In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [6]:
# 1. Create an Image Data Generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    shear_range=0.2,          # Randomly shear images
    zoom_range=0.2,           # Randomly zoom into images
    horizontal_flip=True,     # Randomly flip images horizontally
    validation_split=0.2      # Use 20% of images for validation
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    '../images_binary',             # Directory with training data
    target_size=(775, 616),   # Resize images to 150x150
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary',      # For categorical classification
    subset='training'         # Use the training subset
)

validation_generator = train_datagen.flow_from_directory(
    '../images_binary',             # Directory with validation data
    target_size=(775, 616),
    batch_size=32,
    color_mode='grayscale',
    class_mode='binary',
    subset='validation'       # Use the validation subset
)

# 2. Build the CNN Model
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(775, 616, 1)),
    MaxPooling2D(pool_size=(3, 3)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # For categorical classification
])

# 3. Compile the Model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 4. Define Callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '../models/best_model.keras',          # Save the best model
    monitor='val_accuracy',   # Monitor validation accuracy
    verbose=1,
    save_best_only=True,      # Save only the best model
    mode='max'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',   # Monitor validation accuracy
    patience=5,               # Stop if no improvement after 5 epochs
    verbose=1,
    
    mode='max'
)

callbacks = [checkpoint, early_stopping]

# 5. Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15,                 # Maximum number of epochs
    callbacks=callbacks        # Use the callbacks defined above
)

Found 1222 images belonging to 2 classes.
Found 304 images belonging to 2 classes.
Epoch 1/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7817 - loss: 0.5778
Epoch 1: val_accuracy improved from -inf to 0.86458, saving model to ../models/best_model.keras
38/38 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.7830 - loss: 0.5759 - val_accuracy: 0.8646 - val_loss: 0.4446
Epoch 2/15
 1/38 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.8125 - loss: 0.4817
Epoch 2: val_accuracy did not improve from 0.86458
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8125 - loss: 0.4817 - val_accuracy: 0.7500 - val_loss: 0.5726
Epoch 3/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8589 - loss: 0.4483
Epoch 3: val_accuracy did not improve from 0.86458
38/38 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.8588 - loss: 0.4481 - val_accuracy: 0.8542 - val_loss: 0.4276
Epoch 4/15
 1/38 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.9062 - loss: 0.3258
Epoch 4: val_accuracy improved from 0.86